This code is to generate random math expression with a parameter "max_depth" that defines how deep the compositionality goes. 

In [ ]:
import random
from fractions import Fraction

def gen_expr(num_terms, max_depth=2, current_depth=0, require_nonzero=False,
            digit_low=0, digit_high=9, max_tries=50):
    """
    Generate a random arithmetic expression AND its exact value.
    - require_nonzero: if True, ensure the resulting value != 0 (useful when generating a denominator)
    - digits drawn from [digit_low, digit_high]
    Returns: (expr_str, value_as_Fraction)
    """
    # Base case: single number
    if num_terms == 1 or current_depth >= max_depth:
        if require_nonzero:
            # ensure nonzero leaf for denominators
            d = random.randint(max(digit_low, 1), digit_high)
        else:
            d = random.randint(digit_low, digit_high)
        return str(d), Fraction(d)

    # Try multiple times to satisfy constraints
    for _ in range(max_tries):
        split = random.randint(1, num_terms - 1)
        left_expr, left_val = gen_expr(num_terms, max_depth, current_depth + 1,
                                       require_nonzero=False,
                                       digit_low=digit_low, digit_high=digit_high,
                                       max_tries=max_tries)
        op = random.choice(["+", "-", "*", "/"])
        # If op is division, the right subtree must be nonzero
        right_expr, right_val = gen_expr(num_terms - split, max_depth, current_depth + 1,
                                         require_nonzero=(op == "/"),
                                         digit_low=digit_low, digit_high=digit_high,
                                         max_tries=max_tries)

        # Evaluate
        if op == "+":
            val = left_val + right_val
        elif op == "-":
            val = left_val - right_val
        elif op == "*":
            val = left_val * right_val
        else:  # "/"
            if right_val == 0:
                continue  # should be prevented, but double-check
            val = left_val / right_val

        # If caller needs nonzero (e.g., we are building a denominator), enforce it
        if require_nonzero and val == 0:
            continue

        return f"({left_expr} {op} {right_expr})", val

    # Fallback (very unlikely): produce a simple nonzero/zero constant as needed
    return ("1", Fraction(1)) if require_nonzero else ("0", Fraction(0))



In [ ]:
import ast 
import operator
from fractions import Fraction

Ops = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,  # We'll convert to Fraction-safe division
    # (No Pow, Mod, etc. on purpose)
}

NumberNode = (ast.Constant,) if hasattr(ast, "Num") else ()  # Py<3.8 compatibility

def eval_ast(node: ast.AST) -> Fraction:
    """Recursively evaluate a restricted AST into a Fraction."""
    if isinstance(node, ast.Expression):
        return eval_ast(node.body)

    if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
        # Force integers; but if float shows up, convert via Fraction
        return Fraction(node.value)
    if NumberNode and isinstance(node, NumberNode):
        return Fraction(node.n)

    if isinstance(node, ast.BinOp) and type(node.op) in Ops:
        left = eval_ast(node.left)
        right = eval_ast(node.right)
        if isinstance(node.op, ast.Div):
            if right == 0:
                raise ZeroDivisionError("Division by zero in expression")
            return left / right
        return Ops[type(node.op)](left, right)

    if isinstance(node, ast.UnaryOp) and isinstance(node.op, (ast.UAdd, ast.USub)):
        val = eval_ast(node.operand)
        return +val if isinstance(node.op, ast.UAdd) else -val

    raise ValueError(f"Unsupported expression node: {ast.dump(node)}")

def safe_eval_expr(expr: str) -> Fraction:
    """
    Safely evaluate an arithmetic expression string consisting of
    numbers, +, -, *, /, and parentheses, to a Fraction.
    """
    # Parse in 'eval' mode and walk only allowed nodes
    tree = ast.parse(expr, mode="eval")

    # Validate the AST contains only allowed nodes
    allowed_nodes = (
        ast.Expression, ast.BinOp, ast.UnaryOp, ast.Add, ast.Sub, ast.Mult, ast.Div,
        ast.Constant, ast.Load, ast.USub, ast.UAdd
    ) + NumberNode

    for node in ast.walk(tree):
        if not isinstance(node, allowed_nodes):
            raise ValueError(f"Disallowed syntax: {ast.dump(node)}")

    return eval_ast(tree.body)


/var/folders/xx/d1sg1ttn27b9mx2r1jdfy06c0000gn/T/ipykernel_96296/84598561.py:13: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  NumberNode = (ast.Constant,) if hasattr(ast, "Num") else ()  # Py<3.8 compatibility


In [41]:

expr = [gen_expr(num_terms=10, max_depth=2) for _ in range(50)]
print(expr)
# I want to join with a random operator
final_expr = " + ".join(expr[i][0] for i in range(len(expr)))
print(final_expr)
res = safe_eval_expr(final_expr)
print(res)

[('((5 / 7) - (6 + 2))', Fraction(-51, 7)), ('((0 * 2) * (0 / 5))', Fraction(0, 1)), ('((8 - 5) / (0 - 2))', Fraction(-3, 2)), ('((4 * 7) / (9 + 0))', Fraction(28, 9)), ('((6 + 4) - (4 / 7))', Fraction(66, 7)), ('((4 * 7) - (5 - 5))', Fraction(28, 1)), ('((0 + 1) + 4)', Fraction(5, 1)), ('((6 / 5) - (9 - 5))', Fraction(-14, 5)), ('((0 - 9) - (3 / 3))', Fraction(-10, 1)), ('((8 * 7) * (6 / 1))', Fraction(336, 1)), ('((4 * 8) + 4)', Fraction(36, 1)), ('((1 + 8) - (8 - 5))', Fraction(6, 1)), ('((6 * 7) + (9 + 2))', Fraction(53, 1)), ('((2 + 1) / (2 - 7))', Fraction(-3, 5)), ('((3 / 8) - (0 / 1))', Fraction(3, 8)), ('((8 / 4) - (8 / 5))', Fraction(2, 5)), ('((6 * 2) - (7 * 7))', Fraction(-37, 1)), ('((0 * 6) + (4 + 7))', Fraction(11, 1)), ('((7 * 6) * (8 * 1))', Fraction(336, 1)), ('((4 / 1) + (8 / 5))', Fraction(28, 5)), ('((2 / 2) / (8 - 3))', Fraction(1, 5)), ('((0 / 4) + 0)', Fraction(0, 1)), ('((7 * 5) * (6 / 2))', Fraction(105, 1)), ('((0 / 4) - (5 / 2))', Fraction(-5, 2)), ('((7 + 7

In [34]:
expr2 = [gen_expr(num_terms=1500, max_depth=7)]
print(expr2[0][0])
print(expr2[0][1])
res = safe_eval_expr(expr2[0][0])
print(res)

(((((((4 * 8) - (3 - 7)) + ((9 + 1) + (6 - 8))) / (((4 + 7) * (2 + 1)) / ((4 * 7) - (4 + 0)))) + ((((2 + 5) / (7 - 3)) / ((8 + 1) - (4 - 5))) - (((2 - 5) / (5 + 4)) - ((5 / 2) / (3 + 7))))) - (((((7 + 4) / (4 + 1)) / ((5 * 6) + (8 + 9))) / (((4 - 7) - 0) / 9)) - ((((7 * 1) * (9 + 6)) + ((0 / 7) * (0 * 1))) / 1))) - ((((((6 * 8) + (3 - 9)) - ((0 - 7) / (8 / 2))) + (((4 * 5) - (2 + 4)) - ((3 / 8) * (2 - 8)))) / ((((5 / 8) + (9 - 3)) * ((4 + 3) / (3 - 2))) + (((2 * 3) * (0 - 9)) / ((1 / 2) - (7 - 2))))) / (((((5 + 4) / (6 * 7)) - ((9 * 4) + (7 - 3))) + (((5 + 4) - (8 - 2)) / ((4 - 6) / 7))) - ((((4 * 7) / (3 * 8)) * ((8 - 4) / (1 - 9))) - (((4 + 7) * (6 / 5)) - ((8 * 4) - (0 / 2)))))))
10451016919693/75779361480
10451016919693/75779361480
